In [1]:
import os
import cv2 as cv
import numpy as np
from PIL import Image
from insightface.app import FaceAnalysis
import insightface

Intilize InsightFace

In [10]:
app = FaceAnalysis(name='buffalo_s')
app.prepare(ctx_id=0, det_size=(640, 640))

download_path: /home/sina/.insightface/models/buffalo_s


100%|██████████████████████████████████| 124617/124617 [08:01<00:00, 259.04KB/s]


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/sina/.insightface/models/buffalo_s/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/sina/.insightface/models/buffalo_s/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/sina/.insightface/models/buffalo_s/det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/sina/.insightface/models/buffalo_s/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/sina/.insightface/models/buffalo_s/w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-siz

2025-05-13 18:47:38.079846777 [E:onnxruntime:Default, provider_bridge_ort.cc:2195 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1778 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_cuda.so with error: libcublasLt.so.12: cannot open shared object file: No such file or directory

2025-05-13 18:47:38.323672392 [E:onnxruntime:Default, provider_bridge_ort.cc:2195 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1778 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_cuda.so with error: libcublasLt.so.12: cannot open shared object file: No such file or directory

2025-05-13 18:47:38.334841528 [E:onnxruntime:Default, provider_bridge_ort.cc:2195 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:

Function to load dataset

In [3]:
def load_dataset(dataset_path):
    embeddings = []
    labels = []
    for person_name in os.listdir(dataset_path):
        person_dir = os.path.join(dataset_path, person_name)
        if not os.path.isdir(person_dir):
            continue
        for image_name in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_name)
            try:
                img = Image.open(image_path).convert('RGB')
                img = np.array(img)
                faces = app.get(img)
                if len(faces) > 0:
                    embedding = faces[0].normed_embedding
                    embeddings.append(embedding)
                    labels.append(person_name)
            except Exception as e:
                print(f"Error processing {image_path}: {e}")
    return np.array(embeddings), labels

Function to recognize face

In [4]:
def recognize_face(embedding, dataset_embeddings, dataset_labels, threshold=0.4):
    if len(dataset_embeddings) == 0:
        return "Unknown", False
    
    similarities = np.dot(dataset_embeddings, embedding) / (np.linalg.norm(dataset_embeddings, axis=1) * np.linalg.norm(embedding))
    max_similarities = np.max(similarities)
    if max_similarities > threshold:
        max_index = np.argmax(similarities)
        return dataset_labels[max_index], True
    return "Unknown", False


Load Dataset

In [5]:
dataset_path = "/home/sina/Desktop/Sina/AI/Datasets/Persons images for Face Detection"
dataset_embeddings, dataset_labels = load_dataset(dataset_path)
print(f"Loaded {len(dataset_embeddings)} embeddings from dataset")

Loaded 75 embeddings from dataset


Intialize webcam

In [6]:
cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Error: could not open webcam")
    exit()

In [8]:
# while True:
#     ret, frame = cap.read()
#     if not ret:
#         print("Error: Could not read frame")
#         break
        
#     faces = app.get(frame)
#     for face in faces:
#         bbox = face.normes_embedding
#         x1, y1, x2, y2 = bbox
        
#         embedding = face.normed_embedding
#         name, access_granted = recognize_face(embedding, dataset_embeddings, dataset_labels)
#         if access_granted:
#             text = f"{name} - Access Granted"
#             color = (0, 0, 255)
#         else:
#             text = f"{name} - Access Denied"
#             color = (0, 2550, 0)
            
#         cv.rectangle(frame, (x1, y1), (x2, y2), color, 2)
#         cv.putText(frame, text, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.9. color, 2)
    
#     cv.imshow("Face Recognition", frame)
    
#     if cv.waitKey(1) & 0xFF == ord('q'):
#         break
        
# cap.release()
# cv.destroyALLWindows()


while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame")
        break
        
    faces = app.get(frame)
    for face in faces:
        # اصلاح این خط - استفاده از face.bbox به جای face.normes_embedding
        x1, y1, x2, y2 = face.bbox.astype(int)
        
        embedding = face.normed_embedding
        name, access_granted = recognize_face(embedding, dataset_embeddings, dataset_labels)
        
        if access_granted:
            text = f"{name} - Access Granted"
            color = (0, 255, 0)  # سبز برای دسترسی مجاز
        else:
            text = f"{name} - Access Denied"
            color = (0, 0, 255)  # قرمز برای دسترسی غیرمجاز
            
        cv.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        # اصلاح نقطه به کاما در این خط
        cv.putText(frame, text, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    
    cv.imshow("Face Recognition", frame)
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
# اصلاح حروف بزرگ در این خط
cv.destroyAllWindows()